<a href="https://colab.research.google.com/github/SergioaA64/SergioaA64/blob/main/C%C3%B3pia_de_Aula6_PIB_GeoMunicipios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL para carregar o PIB, População e o Centróide dos municípios

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=QSKD1I6mMdWOPsUp2HUy5aL6wUlftR&prompt=consent&access_type=offline&code_challenge=N9R_AdnMWfj_GpsrD3FPHckar4js21Z8dcKy1N4EvXs&code_challenge_method=S256

Enter verification code: alexpereira


AuthorizationError: ignored

## Pré-processamento dos centroides dos municipios

### Solução 1 - Processamento do arquivo de banco de dados (dbf)




In [ ]:
!pip install dbf

     |████████████████████████████████| 107 kB 4.4 MB/s 
     |████████████████████████████████| 128 kB 30.5 MB/s 


In [ ]:
#Fonte oficial https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27385-localidades.html?=&t=downloads

!wget https://github.com/renatocol/Latitude_Longitude_Brasil/raw/master/BR_Localidades_2010.dbf

--2021-11-23 17:03:14--  https://github.com/renatocol/Latitude_Longitude_Brasil/raw/master/BR_Localidades_2010.dbf
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/renatocol/Latitude_Longitude_Brasil/master/BR_Localidades_2010.dbf [following]
--2021-11-23 17:03:15--  https://raw.githubusercontent.com/renatocol/Latitude_Longitude_Brasil/master/BR_Localidades_2010.dbf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17794056 (17M) [application/octet-stream]
Saving to: ‘BR_Localidades_2010.dbf’

BR_Localidades_2010 100%[===================>]  16.97M   107MB/s    in 0.2s    

2021-11-23 17:03:16 (107 

In [ ]:
import dbf
import pandas as pd

table = dbf.Table(filename='./BR_Localidades_2010.dbf')
table.open(dbf.READ_ONLY)
df = pd.DataFrame(table)
table.close()

print(df)

In [ ]:
df.head()

In [ ]:

# Equivalente SQL: select col9 as cod_ibge, col16 as categoria, col18 as long, col19 as lat from df
df_geo = df[[9,16,18,19]].rename(columns={9:"cod_ibge", 16:"categoria", 18:"long", 19:"lat"})
df_geo['cod_ibge'] = df_geo['cod_ibge'].str.strip()
df_geo['categoria'] = df_geo['categoria'].str.strip()
df_geo = df_geo[df_geo['categoria']=='CIDADE']
df_geo.info()

In [ ]:
df_geo['lat_long'] = df_geo[['lat','long']].apply(lambda x: f"{str(x['lat']).replace(',','.')},{str(x['long']).replace(',','.')}", axis=1)
df_geo.head()

### Solução 2 - Converter o arquivo dbf para xlsx usando o excel. Processar o arquivo xlsx com o pandas
#### Vantagem: mais rápido do que o pandas processar o arquivo dbf.
#### URL do arquivo exportado para xlsx https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx

In [ ]:
df_xlsx = pd.read_excel('https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx', dtype={"CD_GEOCODM,C,20":str})
df_xlsx.head()

In [ ]:
dfxlsx_geo = df_xlsx[['CD_GEOCODM,C,20','NM_CATEGOR,C,50','LONG,N,24,6','LAT,N,24,6']].rename(columns={'CD_GEOCODM,C,20':"cod_ibge", 'NM_CATEGOR,C,50':"categoria", 'LONG,N,24,6':"long", 'LAT,N,24,6':"lat"})
dfxlsx_geo.head()

In [ ]:
# dfxlsx_geo['cod_ibge'] = dfxlsx_geo['cod_ibge'].str.strip()
dfxlsx_geo['categoria'] = dfxlsx_geo['categoria'].str.strip()
dfxlsx_geo = dfxlsx_geo[dfxlsx_geo['categoria']=='CIDADE']
dfxlsx_geo['lat_long'] = dfxlsx_geo[['lat','long']].apply(lambda x: f"{str(x['lat']).replace(',','.')},{str(x['long']).replace(',','.')}", axis=1)
dfxlsx_geo.head()

## Join com a tabela de PIB per capita

In [ ]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
# project_id = 'enap-331414' # Defina o id do seu projeto no bigquery!!!!!
  project_id = 'enap-sao00001' # Defina o id do seu projeto no bigquery!!!!!## Defina o id do seu projeto no bigquery!!!!!

df_pibpercapita = pd.io.gbq.read_gbq('''
SELECT pop.*, dsc.nome_municipio, pib.pib, pib.pib/pop.populacao as pibpercapita FROM `basedosdados.br_ibge_populacao.municipio` pop
LEFT JOIN `basedosdados.br_ibge_pib.municipio` pib on pop.id_municipio = pib.id_municipio and pib.ano = pop.ano
LEFT JOIN (
    select distinct (sc.id_municipio), sc.nome_municipio from `basedosdados.br_geobr_mapas.setor_censitario_2010` sc
    ) as dsc on dsc.id_municipio = pop.id_municipio
''', project_id=project_id)

df_pibpercapita.head()

,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita
0,1991,RO,1100015,31981.0,Alta Floresta D'oeste,NaN,NaN
1,1992,RO,1100015,34768.0,Alta Floresta D'oeste,NaN,NaN
2,1993,RO,1100015,37036.0,Alta Floresta D'oeste,NaN,NaN
3,1994,RO,1100015,39325.0,Alta Floresta D'oeste,NaN,NaN
4,1995,RO,1100015,41574.0,Alta Floresta D'oeste,NaN,NaN


In [ ]:
#Algebra relacional: Join. Ou seja, junção de tabelas.
df_merge = df_pibpercapita.merge(df_geo[['cod_ibge','lat_long']], how='left', left_on='id_municipio', right_on='cod_ibge') 
df_merge.head()

,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita,cod_ibge,lat_long
0,1991,RO,1100015,31981.0,Alta Floresta D'oeste,NaN,NaN,1100015,"-11.9355403048,-61.9998238963"
1,1992,RO,1100015,34768.0,Alta Floresta D'oeste,NaN,NaN,1100015,"-11.9355403048,-61.9998238963"
2,1993,RO,1100015,37036.0,Alta Floresta D'oeste,NaN,NaN,1100015,"-11.9355403048,-61.9998238963"
3,1994,RO,1100015,39325.0,Alta Floresta D'oeste,NaN,NaN,1100015,"-11.9355403048,-61.9998238963"
4,1995,RO,1100015,41574.0,Alta Floresta D'oeste,NaN,NaN,1100015,"-11.9355403048,-61.9998238963"


In [ ]:
del df_merge['cod_ibge']
df_merge.head()

,ano,sigla_uf,id_municipio,populacao,nome_municipio,pib,pibpercapita,lat_long
0,1991,RO,1100015,31981.0,Alta Floresta D'oeste,NaN,NaN,"-11.9355403048,-61.9998238963"
1,1992,RO,1100015,34768.0,Alta Floresta D'oeste,NaN,NaN,"-11.9355403048,-61.9998238963"
2,1993,RO,1100015,37036.0,Alta Floresta D'oeste,NaN,NaN,"-11.9355403048,-61.9998238963"
3,1994,RO,1100015,39325.0,Alta Floresta D'oeste,NaN,NaN,"-11.9355403048,-61.9998238963"
4,1995,RO,1100015,41574.0,Alta Floresta D'oeste,NaN,NaN,"-11.9355403048,-61.9998238963"


In [ ]:
project_id = 'enap-331414'

In [ ]:
df_merge.to_gbq("enapdatasets.pibpercapita",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

5it [00:20,  4.06s/it]
